# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.232947980096                   -0.50    7.0         
  2   -7.249812696224       -1.77       -1.39    1.0   7.66ms
  3   -7.250997497497       -2.93       -1.87    2.0   9.15ms
  4   -7.251132158447       -3.87       -1.98    2.0   8.79ms
  5   -7.251317357067       -3.73       -2.48    1.0   7.58ms
  6   -7.251337217153       -4.70       -3.12    1.0   7.77ms
  7   -7.251338440579       -5.91       -3.25    3.0   10.3ms
  8   -7.251338746671       -6.51       -3.70    1.0   7.98ms
  9   -7.251338794219       -7.32       -4.28    2.0   9.57ms
 10   -7.251338798380       -8.38       -4.70    2.0   9.74ms
 11   -7.251338798625       -9.61       -5.01    1.0   7.97ms
 12   -7.251338798679      -10.27       -5.40    1.0   8.16ms
 13   -7.251338798703      -10.62       -5.97    2.0   9.46ms
 14   -7.251338798704      -11.81       -6.45    1.0   8.73ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.11125200227717003
[ Info: Arnoldi iteration step 2: normres = 0.5359627532090409
[ Info: Arnoldi iteration step 3: normres = 0.5971402346599269
[ Info: Arnoldi iteration step 4: normres = 0.24209941364441911
[ Info: Arnoldi iteration step 5: normres = 0.605717465441287
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.66e-02, 9.21e-02, 5.50e-01, 2.36e-01, 8.37e-03)
[ Info: Arnoldi iteration step 6: normres = 0.2220893362129997
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.81e-03, 1.03e-01, 1.30e-01, 1.07e-01, 9.95e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06149429593255662
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (7.95e-05, 5.40e-03, 6.42e-03, 2.78e-02, 4.43e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09713051894932645
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.33e-0